# ห้องปฏิบัติการเสริม: การถดถอยเชิงเส้น (Linear Regression) ด้วย Scikit-Learn

มีเครื่องมือเรียนรู้ของเครื่องแบบโอเพนซอร์สที่สามารถใช้ในเชิงพาณิชย์ได้ ชื่อว่า  [scikit-learn](https://scikit-learn.org/stable/index.html)    เครื่องมือชุดนี้ประกอบด้วยการใช้งานของอัลกอริทึมหลายอย่างที่คุณจะได้ทำงานด้วยในหลักสูตรนี้


## เป้าหมาย
ในห้องปฏิบัติการนี้ คุณจะ:

- ใช้งาน scikit-learn เพื่อนำไปใช้ในการถดถอยเชิงเส้น (linear regression) โดยใช้การไล่ระดับ (Gradient Descent)


## เครื่องมือ
คุณจะใช้ฟังก์ชันจาก scikit-learn รวมถึง matplotlib และ NumPy

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_multi.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/data/houses.txt'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_common.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)


response = requests.get(url2)
with open('lab_utils_multi.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
data_path = Path('data')
if not data_path.is_dir():
  data_path.mkdir(parents=True, exist_ok=True)
with open('data/houses.txt', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from lab_utils_multi import  load_house_data
from lab_utils_common import dlc
np.set_printoptions(precision=2)
plt.style.use('./deeplearning.mplstyle')

# การไล่ระดับ (Gradient Descent)
Scikit-learn มีโมเดลการถดถอย (regression) ที่ใช้การไล่ระดับ (gradient descent) [sklearn.linear_model.SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#examples-using-sklearn-linear-model-sgdregressor). เช่นเดียวกับการใช้งาน gradient descent ครั้งก่อน ๆ โมเดลนี้ทำงานได้ดีที่สุดเมื่ออินพุตถูกปรับมาตรฐาน (normalized) [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) เป็นเทคนิคการแปลงข้อมูลให้เป็นค่ามาตรฐาน z-score normalization เหมือน lab ก่อนหน้านี้ แต่นี่จะเรียกว่า 'standard score'.

### โหลด data set

In [ ]:
X_train, y_train = load_house_data()
X_features = ['size(sqft)','bedrooms','floors','age']

### มาตรฐานข้อมูลการฝึก (Scale/normalize the training data)

In [ ]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_train)
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

### สร้างและปรับโมเดลการถดถอย (Regression Model)

In [ ]:
sgdr = SGDRegressor(max_iter=1000)
sgdr.fit(X_norm, y_train)
print(sgdr)
print(f"number of iterations completed: {sgdr.n_iter_}, number of weight updates: {sgdr.t_}")

### ดูค่าพารามิเตอร์
หมายเหตุ: ค่าพารามิเตอร์ที่แสดงเป็นค่าที่เกี่ยวข้องกับข้อมูลที่ถูกปรับมาตรฐานแล้ว (normalized input data) ค่าพารามิเตอร์ที่ได้ใกล้เคียงมากกับค่าพารามิเตอร์ที่พบในแล็บก่อนหน้านี้กับข้อมูลชุดนี้


In [ ]:
b_norm = sgdr.intercept_
w_norm = sgdr.coef_
print(f"model parameters:                   w: {w_norm}, b:{b_norm}")
print( "model parameters from previous lab: w: [110.56 -21.27 -32.71 -37.97], b: 363.16")

### การทำนายค่าเป้าหมาย (Make predictions)

คาดการณ์เป้าหมายของข้อมูลฝึกสอน ใช้ทั้งฟังก์ชัน `predict` และการคำนวณโดยใช้  $w$ และ $b$

In [ ]:
# make a prediction using sgdr.predict()
y_pred_sgd = sgdr.predict(X_norm)
# make a prediction using w,b. 
y_pred = np.dot(X_norm, w_norm) + b_norm  
print(f"prediction using np.dot() and sgdr.predict match: {(y_pred == y_pred_sgd).all()}")

print(f"Prediction on training set:\n{y_pred[:4]}" )
print(f"Target values \n{y_train[:4]}")

### พล็อตผลลัพธ์
มาพล็อตค่าทำนายเทียบกับค่าเป้าหมายกันครับ

In [ ]:
# plot predictions and targets vs original features    
fig,ax=plt.subplots(1,4,figsize=(12,3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'target')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],y_pred,color=dlc["dlorange"], label = 'predict')
ax[0].set_ylabel("Price"); ax[0].legend();
fig.suptitle("target versus prediction using z-score normalized model")
plt.show()

## ยินดีด้วย!
ในแล็บนี้ คุณได้:
- ใช้งานเครื่องมือเรียนรู้ของเครื่องแบบโอเพนซอร์ส scikit-learn
- นำการถดถอยเชิงเส้น (linear regression) ไปใช้โดยใช้การไล่ระดับ (gradient descent) และการทำให้ลักษณะเป็นปกติ (feature normalization) จากเครื่องมือดังกล่าว






